In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from lightfm import LightFM

data = pd.read_csv('preprocessed.csv')
sample_size = 20000
if len(data) > sample_size:
    data = data.sample(n=sample_size, random_state=42)

unique_users = data['customer_id'].unique()
unique_items = data['product_id'].unique()
user_map = {id: idx for idx, id in enumerate(unique_users)}
item_map = {id: idx for idx, id in enumerate(unique_items)}
data['user_idx'] = data['customer_id'].apply(lambda x: user_map[x])
data['item_idx'] = data['product_id'].apply(lambda x: item_map[x])

num_users = len(unique_users)
num_items = len(unique_items)
row = data['user_idx']
col = data['item_idx']
rating = data['star_rating']
input_matrix = csr_matrix((rating, (row, col)), shape=(num_users, num_items))

In [3]:
input_matrix

<19705x12427 sparse matrix of type '<class 'numpy.int64'>'
	with 19997 stored elements in Compressed Sparse Row format>

In [4]:
model = LightFM(loss='warp')
batch_size = 1000
num_epochs = 10

num_samples = input_matrix.shape[0]
for epoch in range(num_epochs):
    idx = np.random.permutation(num_samples)
    input_matrix_shuffled = input_matrix[idx, :]
    
    for i in range(0, num_samples, batch_size):
        batch = input_matrix_shuffled[i:i+batch_size, :]
        model.fit_partial(batch, epochs=1)
        
    train_auc = auc_score(model, input_matrix).mean()
    print("Epoch {}/{}: train AUC = {:.4f}".format(epoch+1, num_epochs, train_auc))

: 

: 